In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
os.chdir('/content/drive/MyDrive/Colab Notebooks/Hackerton')

In [3]:
!pip install --upgrade wandb -qq

     |████████████████████████████████| 1.8 MB 12.4 MB/s 
     |████████████████████████████████| 145 kB 56.2 MB/s 
     |████████████████████████████████| 181 kB 62.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [4]:
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import os, sys
import numpy as np
from pathlib import Path
import pandas as pd
from pprint import pprint
import easydict
import wandb
import torch

from lane_change_risk_detection.dataset import DataSet
from lane_change_risk_detection.models import LSTM_Classifier, CNN_LSTM_Classifier, CNN_Classifier, ResNet50_LSTM_Classifier 
from lane_change_risk_detection.train import Trainer

PROJECT_NAME = "socar_hackerton"

In [20]:
args_parsed = easydict.EasyDict({
  'input_path': '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/masked_images',
	'pkl_path':'/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_dataset.pkl',
	'load_pkl': True,
	'save_pkl_path':'/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_dataset.pkl',
	'save_pkl': True,
  'model_name': 'cnn_lstm',
  'n_folds': 1,
  'train_ratio': 0.9,
  'downsample': False,
  'seed': 0,
  'test_step': 10,
  'device': 'cuda',
  'epochs': 10,
  'batch_size': 32,
  'bnorm': False,
  'dropout': 0.1,
  'learning_rate': 3e-5,
  'weight_decay': 5e-4 

   })
class Config:

  def __init__(self, args):
    self.args_parsed = args
    self.wandb = wandb.init(project=PROJECT_NAME)
    self.wandb_config = self.wandb.config

    for arg_name in vars(self.args_parsed):
      self.__dict__[arg_name] = getattr(self.args_parsed, arg_name)
      self.wandb_config[arg_name] = getattr(self.args_parsed, arg_name)
    self.wandb_base_dir = Path('/content/drive/MyDrive/Colab Notebooks/Hackerton/data/masked_images').resolve

def load_dataset(raw_image_path, risk_path, config=None):
  dataset = DataSet()
  dataset.read_video(raw_image_path, option='fixed frame amount', number_of_frames = 50, scaling = 'scale', scale_x = 0.1, scale_y = 0.1)
  dataset.read_risk_data(risk_path)
  dataset.convert_risk_to_one_hot(risk_threshold = 0.05)
  if config != None and config.save_pkl:
    parent_path = '/'.join(config.save_pkl_path.split('/')[:-1]) + '/'
    fname = config.save_pkl_path.split('/')[-1]
    dataset.save(save_dir = parent_path, filename=fname)
    print('Saved pickled dataset')
  return dataset


def load_pickle(pkl_path):
	dataset = DataSet().loader(str(pkl_path))
	return dataset



def reshape_dataset(dataset):
	'''
		input -> (batch, frames, height, width, channels)
		output -> (batch, frames, channels, height, width)
	'''
	return np.swapaxes(np.swapaxes(dataset, -1, -3), -1, -2)

def train_model(dataset, config):
	pth_path = '/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_parameter.pth'
	dataset.video = reshape_dataset(dataset.video)
	video_sequences = dataset.video
	labels = dataset.risk_one_hot
	clip_names = np.array(['default_all']*len(video_sequences))
	if hasattr(dataset, 'foldernames'): 
		clip_names = np.concatenate((clip_names, dataset.foldernames), axis=0)

	if config.model_name == 'gru':
		model = LSTM_Classifier(video_sequences.shape, 'gru', config)
	elif config.model_name == 'lstm':
		model = LSTM_Classifier(video_sequences.shape, 'lstm', config)
	elif config.model_name == 'cnn':
		model = CNN_Classifier(video_sequences.shape, config)
	elif config.model_name == 'cnn_lstm':
		model = CNN_LSTM_Classifier(video_sequences.shape, config)
	elif config.model_name == 'resnet': 
		model = ResNet50_LSTM_Classifier(video_sequences.shape, config)
	
	trainer = Trainer(config)
	trainer.init_dataset(video_sequences, labels, clip_names)
	trainer.build_model(model)
 
	if config.n_folds > 1:
		trainer.train_n_fold_cross_val()
	else:
		trainer.train_model()
	torch.save(model.state_dict(), pth_path)
 

config = Config(args_parsed)
image_path = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/masked_images'
risk = '/content/drive/MyDrive/Colab Notebooks/Hackerton/test/LCTable.csv'

if config.load_pkl:
	dataset = load_pickle(Path(config.pkl_path).resolve())
else:
	dataset = load_dataset(image_path, risk, config=config)
#dataset = load_dataset(image_path, risk, config=config);
	
# train model
model = train_model(dataset, config)

Total dataset breakdown: {'all': 163}
Number of Training Sequences Included:  146
Number of Testing Sequences Included:  17
Num of Training Labels in Each Class: [139   7], Class Weights: tensor([ 0.5252, 10.4286], dtype=torch.float64)
Num of Testing Labels in Each Class: [16  1], Class Weights: tensor([ 0.5252, 10.4286], dtype=torch.float64)


Epoch: 0000, loss_train: 0.6959:   0%|          | 0/10 [00:12<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch: 0000, loss_train: 0.6959:  10%|█         | 1/10 [00:13<02:00, 13.43s/it]


train loss: 0.6783384713408065, acc: 0.952054794520548 [[139   0], [  7   0]] mcc: 0.0 
test loss: 0.7177019715309143, acc: 0.9411764705882353 [[16  0], [ 1  0]] mcc: 0.0


Epoch: 0009, loss_train: 0.6251: 100%|██████████| 10/10 [02:03<00:00, 12.35s/it]


In [61]:
im_path = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/test/'

dataset = DataSet()
dataset.read_video(im_path, option='fixed frame amount', number_of_frames = 50, scaling = 'scale', scale_x = 0.1, scale_y = 0.1)
dataset.video = reshape_dataset(dataset.video)
video_sequences = dataset.video

#cnn_lstm = CNN_LSTM_Classifier(video_sequences.shape, config)
#cnn_lstm.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_parameter.pth'))
model = CNN_LSTM_Classifier(video_sequences.shape, config)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_parameter.pth'))

100%|██████████| 1/1 [00:14<00:00, 14.79s/it]


<All keys matched successfully>

In [62]:
model.eval()

CNN_LSTM_Classifier(
  (dropout): Dropout(p=0.1, inplace=False)
  (bn1): BatchNorm3d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm3d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm3d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (c1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (c2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=49200, out_features=200, bias=True)
  (l2): Linear(in_features=200, out_features=50, bias=True)
  (lstm1): LSTM(50, 20, batch_first=True)
  (l3): Linear(in_features=20, out_features=2, bias=True)
)

In [66]:
type(video_sequences)
#model.forward(video_sequences)

numpy.ndarray

In [75]:
video_sequences.shape

(1, 50, 3, 86, 154)

In [67]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [77]:
torch_video_sequences = torch.Tensor(video_sequences)
type(torch_video_sequences)

torch.Tensor

In [79]:
with torch.no_grad():
    model.eval()
    inputs = torch_video_sequences
    outputs = model(inputs)
    print(outputs)

tensor([ 0.3598, -0.1974])
